<a href="https://colab.research.google.com/github/heriyang9000/test2/blob/main/test1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
# import getpass
import pandas as pd
# import ipywidgets as widgets
import panel as pn
import requests
import datetime as dt
import numpy as np
import param


In [87]:
pn.extension()
api_key = pn.widgets.PasswordInput(name='API Key', placeholder='Enter Your API Key')
api_key

PasswordInput(name='API Key', placeholder='Enter Your API Key')

In [88]:
if api_key.value == "":
  raise Exception("You have not input your API key !!!")
else:
  response = requests.get(f"https://api.quantxi.com/user?api_key={api_key.value}")  
  print(response.json())

{'status': 'success', 'data': {'user_id': 2, 'nama': 'heri', 'sekolah': 'smk', 'tgl_created': '2020-11-10', 'tgl_expired': '2020-11-17', 'last_akses': '2020-11-10', 'total_play': 0}}


In [49]:
pn.extension()
class TestSetting(param.Parameterized):
    initial_capital = param.Integer(default=1000000, bounds=(1000000,100000000), step=100000)
    Spread_Slippage = param.Number(default=0.0100, bounds=(0.001,0.0500), step=0.01)
    Commision = param.Number(default=0.0100, bounds=(0.001,0.0500), step=0.01)
    Interest = param.Number(default=0.0100, bounds=(0.001,0.0500), step=0.01)
    
test_setting = TestSetting()
pn.Param(test_setting)

Param(TestSetting)

In [6]:
pn.extension()
exchange = pn.widgets.Select(name='Select Exchange', options=['NASDAQ', 'NYSE', 'LSE', 'EURONEXT'])
exchange

Select(options=['NASDAQ', 'NYSE', ...], value='NASDAQ')

In [92]:
# pn.extension()
# cross_selector = pn.widgets.CrossSelector(name='Fruits', value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'])
# cross_selector
# ticker1 = pn.widgets.MultiSelect(name='Select Tickers', value=[], options=['AAPL', 'AMD', 'AMZN', 'ATN'], size=8)

pn.extension()
ticker1 = pn.widgets.CrossSelector(name='Select Tickers', value=[], options=['AAPL', 'AMD', 'AMZN', 'ATN'], size=8)
ticker2 = pn.widgets.CrossSelector(name='Select Tickers', value=[], options=['AAPL2', 'AMD2', 'AMZN2', 'ATN2'], size=8)
ticker3 = pn.widgets.CrossSelector(name='Select Tickers', value=[], options=['AAPL3', 'AMD3', 'AMZN3', 'ATN3'], size=8)
ticker4 = pn.widgets.CrossSelector(name='Select Tickers', value=[], options=['AAPL4', 'AMD4', 'AMZN4', 'ATN4'], size=8)
if exchange.value == 'NASDAQ':
    display(ticker1)
elif exchange.value == 'NYSE':
    display(ticker2)
elif exchange.value == 'LSE':
    display(ticker3)
elif exchange.value == 'EURONEXT':
    display(ticker4)
else:
  print("no exchange select")
 

CrossSelector(name='Select Tickers', options=['AAPL', 'AMD', ...], size=8)

In [94]:
print(ticker1.value[0])

AAPL


In [ ]:
if len(ticker1.value) < 4:
  raise Exception("You dont have enough tickers")
else:
  df01 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df02 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df03 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df04 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df05 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df06 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df07 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df08 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df09 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df10 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df11 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df12 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df13 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df14 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df15 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df16 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df17 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df18 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df19 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df20 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df21 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df22 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df23 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df24 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df25 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df26 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df27 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df28 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df29 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")
  df30 = pd.read_csv(f"https://raw.githubusercontent.com/heriyang9000/mysys/main/dataset/NASDAQ/{ticker1.value[0]}.csv")

  print(df01.to_string()) 

In [25]:
widgets.Combobox(
    # value='John',
    placeholder='Choose Someone',
    options=['Paul', 'John', 'George', 'Ringo'],
    description='Combobox:',
    ensure_option=True,
    disabled=False
)

Combobox(value='', description='Combobox:', ensure_option=True, options=('Paul', 'John', 'George', 'Ringo'), p…

In [28]:
pn.extension()
progress = pn.indicators.Progress(name='Progress', value=20, width=200)
progress

Progress(value=20, width=200)

In [52]:
pn.extension()

terminal = pn.widgets.Terminal(
    "Welcome to the Panel Terminal!\nI'm based on xterm.js\n\n",
    options={"cursorBlink": True},
    height=300, sizing_mode='stretch_width'
)

terminal

AttributeError: ignored

In [4]:
pn.extension()
large_df = pd._testing.makeCustomDataframe(100, 5) 

paginated_table = pn.widgets.Tabulator(large_df, disabled=True, pagination='remote', page_size=10)
paginated_table

Tabulator(disabled=True, page_size=10, pagination='remote', value=C0       C_l0_g0 C...)

In [3]:
pn.extension()
df = pd.DataFrame({
    'int': [1, 2, 3],
    'float': [3.14, 6.28, 9.42],
    'str': ['A', 'B', 'C'],
    'bool': [True, False, True],
    'date': [dt.date(2019, 1, 1), dt.date(2020, 1, 1), dt.date(2020, 1, 10)]
}, index=[1, 2, 3])

df_widget = pn.widgets.Tabulator(df, disabled =True, configuration={"headerSort": False, "resizableColumns": False})
df_widget.servable()

Tabulator(disabled=True, value=   int  float s...)

In [10]:
pn.extension()
json_obj = {   
    'data_id': "00000",
    'timestamp': "mm/dd/yyy 00:00:00",
    'buying_power': "No Data",
    'stock01_price': "No Data",
    'stock02_price': "No Data",
    'stock03_price': "No Data",
    'stock04_price': "No Data",
    'stock05_price': "No Data",
    'stock06_price': "No Data",
    'stock07_price': "No Data",
    'stock08_price': "No Data",
    'stock09_price': "No Data",
    'stock10_price': "No Data",
    'stock11_price': "No Data",
    'stock12_price': "No Data",
    'stock13_price': "No Data",
    'stock14_price': "No Data",
    'stock15_price': "No Data",
    'stock16_price': "No Data",
    'stock17_price': "No Data",
    'stock18_price': "No Data",
    'stock19_price': "No Data",
    'stock20_price': "No Data",
    'stock21_price': "No Data",
    'stock22_price': "No Data",
    'stock23_price': "No Data",
    'stock24_price': "No Data",
    'stock25_price': "No Data",
    'stock26_price': "No Data",
    'stock27_price': "No Data",
    'stock28_price': "No Data",
    'stock29_price': "No Data",
    'stock30_price': "No Data"
}

json = pn.pane.JSON(json_obj, name='JSON', height=300, width=500)

json

JSON(dict, height=300, sizing_mode='fixed', width=500)